In [1]:
import sys
sys.path.append('./lib')

from DBConnection import DBConnection
from ReadJsonConfig import read_json_config
import UsefulFunctions

### example code:

``` python
# the `param_name=value` syntax is optional but useful when reading &/or reviewing the code.
UsefulFunctions.duplicate_table(
    db_source=DBConnection(read_json_config('./secrets/db_config_dev.json')),
    db_dest=DBConnection(
        read_json_config('./secrets/db_config_localhost.json'), 
        editable_environment=True
    ),
    schema_name='public', 
    table_name='inventory_forecast_store',
    truncate_table=True
)
```

In [2]:
schema = read_json_config('outputs/schemas.json')
filter_keywords = [
    "analytics",
    'market',
    'backup',
    'finance',
    'forecast',
    'customer',
    'call',
    'sales',
    'date',
    'time',
    'acct',
    'account',
    'evo',
    'dev',
    'test'
]

filtered_schemas =  [key for key in schema.keys() if not any([keyword in key.lower() for keyword in filter_keywords])]
for schema_name in ['bishfix', 'bishnet']:
    try:
        print("duplicating schema: ", schema_name)
        UsefulFunctions.duplicate_schema(
            db_source=DBConnection(read_json_config('secrets/db_config_prod.json')),
            db_dest=DBConnection(
                read_json_config('secrets/db_config_localhost.json'),
                editable_environment=True
            ),
            schema_name=schema_name,
            truncate_tables=True,
            filter_keywords=filter_keywords
        )
    except Exception as e:
        print(e)
        print(f"failed to duplicate schema: {schema_name}")
        print('')

duplicating schema:  nada
source:       moneyBall_DB  :  moneyballdev.cdyzkfxmqyqc.us-west-2.rds.amazonaws.com
destination:  postgres  :  localhost
awaiting confirmation: 
awaiting confirmation: 
  > see the prompt above to confirm that you want to: TRUNCATE destination tables & DUPLICATE the schema

execution aborted...

closing DB connection : moneyballdev.cdyzkfxmqyqc.us-west-2.rds.amazonaws.com
closing DB connection : localhost
duplicating schema:  public
source:       moneyBall_DB  :  moneyballdev.cdyzkfxmqyqc.us-west-2.rds.amazonaws.com
destination:  postgres  :  localhost
awaiting confirmation: 
awaiting confirmation: 
  > see the prompt above to confirm that you want to: TRUNCATE destination tables & DUPLICATE the schema

truncating existing destination tables & duplicating schema records:  public
Schema public created successfully.
existing tables:  ['inventory_order_audit_cancel', 'inventory_flooring_order', 'roamly_vehicletype_map', 'bishnet_service_check_for_approval', 'inv